In [1]:
import glob
import logging
import os
import random
import shutil
import json

from typing import Dict, List, Tuple
from argparse import Namespace

import numpy as np
import sklearn

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

from tqdm import tqdm, trange
from tqdm.notebook import tqdm

from transformers import (
    AdamW,
    AutoConfig,
    AutoTokenizer,
    AutoModel,
    BertForMultipleChoice,
    RobertaForMultipleChoice,
    XLNetForMultipleChoice,
    GPT2LMHeadModel,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

from dataset import (
    ResponseGenerationDataset,
    KnowledgeSelectionDataset,
    SPECIAL_TOKENS
)

from utils.argument import (
    set_default_params,
    set_default_dataset_params,
    update_additional_params,
    verify_args
)

from utils.data import write_selection_preds, write_detection_preds

logger = logging.getLogger(__name__)

## Set arguments for dataset, model, and training

In [2]:
from argparse import _AttributeHolder
class Namespace(_AttributeHolder):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    def __str__(self):
        return self

In [3]:
args = Namespace(
    checkpoint = None,
    dataroot = 'data',
    device = "cuda" if torch.cuda.is_available() else "cpu",
    eval_all_snippets = False,
    eval_dataset = 'val',
    eval_desc = '',
    eval_only = False,
    exp_name = 'selection',
    history_max_tokens = -1,
    knowledge_file = 'knowledge.json',
    knowledge_max_tokens = -1,
    labels_file = None,
    local_rank = -1,
    model_name_or_path = 'bert-base-uncased',
    model_type = '',
    negative_sample_method = '',
    no_labels = False,
    output_dir = "outputs",
    output_file = 'pred/val/selection.json',
    params_file = "configs/selection_params.json",
    relevance_ratio = '1',
    task_dataset = ''
    )

In [4]:
with open(args.params_file, "r") as f:
    params = json.load(f)
    args = vars(args)

    update_additional_params(params, args)
    args.update(params)
    args = Namespace(**args)

In [5]:
args.params = params # used for saving checkpoints
set_default_params(args)
dataset_args = Namespace(**args.dataset_args)
set_default_dataset_params(dataset_args)
dataset_args.local_rank = args.local_rank
dataset_args.task = args.task

In [6]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# args.n_gpu = torch.cuda.device_count()
# args.device = device

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
args.n_gpu = 1
args.device = device

In [8]:
args

Namespace(adam_epsilon=1e-08, checkpoint=None, dataroot='data', dataset_args={'n_candidates': 2, 'eval_all_snippets': False, 'negative_sample_method': 'oracle', 'history_max_utterances': 1000000, 'history_max_tokens': 128, 'knowledge_max_tokens': 128, 'dataroot': 'data', 'knowledge_file': 'knowledge.json'}, device=device(type='cuda', index=1), eval_all_snippets=False, eval_dataset='val', eval_desc='', eval_only=False, exp_name='selection', fp16='', gradient_accumulation_steps=8, history_max_tokens=-1, knowledge_file='knowledge.json', knowledge_max_tokens=-1, labels_file=None, learning_rate=6.25e-05, local_rank=-1, max_candidates_per_forward_eval=16, max_grad_norm=1.0, model_name_or_path='bert-base-uncased', model_type='', n_gpu=1, negative_sample_method='', no_labels=False, num_train_epochs=2, output_dir='outputs', output_file='pred/val/selection.json', params={'dataset_args': {'n_candidates': 2, 'eval_all_snippets': False, 'negative_sample_method': 'oracle', 'history_max_utterances': 

In [9]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

## Load the pre-trained model

In [10]:
config = AutoConfig.from_pretrained(args.model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
tokenizer.add_special_tokens(SPECIAL_TOKENS)

7

## Load dataset

In [11]:
train_dataset = KnowledgeSelectionDataset(dataset_args, tokenizer, split_type="train")
eval_dataset = KnowledgeSelectionDataset(dataset_args, tokenizer, split_type="val")

## Train

### Train and eval function

In [12]:
def train(args, train_dataset, eval_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, run_batch_fn_train, run_batch_fn_eval) -> Tuple[int, float]:
    if args.local_rank in [-1, 0]:
        log_dir = os.path.join("outputs", args.exp_name) if args.exp_name else "outputs"
        tb_writer = SummaryWriter(log_dir)
        args.output_dir = log_dir

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=args.train_batch_size,
        collate_fn=train_dataset.collate_fn
    )

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    global_step = 0
    model.zero_grad()
    train_iterator = trange(
        0, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # for reproducibility

    for _ in train_iterator:
        local_steps = 0
        tr_loss = 0.0
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            model.train()
            loss, _, _, _ = run_batch_fn_train(args, model, batch)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()

            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1
                local_steps += 1
                epoch_iterator.set_postfix(Loss=tr_loss/local_steps)

        results = evaluate(args, eval_dataset, model, tokenizer, run_batch_fn_eval, desc=str(global_step))
        if args.local_rank in [-1, 0]:
            for key, value in results.items():
                tb_writer.add_scalar("eval_{}".format(key), value, global_step)
            tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
            tb_writer.add_scalar("loss", tr_loss / local_steps, global_step)

            checkpoint_prefix = "checkpoint"
            # Save model checkpoint
            output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
            os.makedirs(output_dir, exist_ok=True)
            model_to_save = (
                model.module if hasattr(model, "module") else model
            )  # Take care of distributed/parallel training

            logger.info("Saving model checkpoint to %s", output_dir)
            model_to_save.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)

            torch.save(args, os.path.join(output_dir, "training_args.bin"))
            with open(os.path.join(output_dir, "params.json"), "w") as jsonfile:
                json.dump(args.params, jsonfile, indent=2, default=lambda x: str(x))
            logger.info("Saving model checkpoint to %s", output_dir)

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / local_steps


In [13]:
def evaluate(args, eval_dataset, model, tokenizer, run_batch_fn, desc="") :
    if args.local_rank in [-1, 0]:
        eval_output_dir = args.output_dir
        os.makedirs(eval_output_dir, exist_ok=True)

    # eval_batch_size for selection must be 1 to handle variable number of candidates
    if args.task == "selection":
        args.eval_batch_size = 1
    else:
        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset,
        sampler=eval_sampler,
        batch_size=args.eval_batch_size,
        collate_fn=eval_dataset.collate_fn
    )


    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()
    data_infos = []
    all_preds = []
    all_labels = []

    for batch in tqdm(eval_dataloader, desc="Evaluating", disable=args.local_rank not in [-1, 0]):
        with torch.no_grad():
            loss, lm_logits, mc_logits, mc_labels = run_batch_fn(args, model, batch)
            if args.task == "selection":
                data_infos.append(batch[-1])
            all_preds.append(mc_logits.detach().cpu().numpy())
            all_labels.append(mc_labels.detach().cpu().numpy())
            eval_loss += loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps

    if args.task.lower() == "generation":
        perplexity = torch.exp(torch.tensor(eval_loss))
        result = {"perplexity": perplexity, "loss": eval_loss}
    elif args.task.lower() == "selection":
        all_labels = np.array(all_labels).reshape(-1)
        all_pred_ids = np.array([np.argmax(logits) for logits in all_preds])
        accuracy = np.sum(all_pred_ids == all_labels) / len(all_labels)
        logger.info("Avg. # of candidates: %f", sum([len(arr[0]) for arr in all_preds]) / len(all_preds))
        result = {"loss": eval_loss, "accuracy": accuracy}
        if args.output_file:
            sorted_pred_ids = [np.argsort(logits.squeeze())[::-1] for logits in all_preds]
            write_selection_preds(eval_dataset.dataset_walker, args.output_file, data_infos, sorted_pred_ids, topk=5)
    else:
        raise ValueError("args.task not in ['generation', 'selection', 'detection'], got %s" % args.task)

    if args.local_rank in [-1, 0]:
        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "a") as writer:
            logger.info("***** Eval results %s *****" % desc)
            writer.write("***** Eval results %s *****\n" % desc)
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return result

### Functions to train or evaluate for each batch

In [14]:
def run_batch_selection_train(args, model, batch):
    batch = tuple(input_tensor.to(args.device) for input_tensor in batch if isinstance(input_tensor, torch.Tensor))
    input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels = batch
    model_outputs = model(
        input_ids=input_ids,
        labels=mc_labels
    )
    loss, mc_logits = model_outputs[:2]
    return loss, torch.tensor([]) , mc_logits, mc_labels

def run_batch_selection_eval(args, model, batch):
    candidates_per_forward = args.max_candidates_per_forward_eval * (args.n_gpu if isinstance(model, torch.nn.DataParallel) else 1)
    batch = tuple(input_tensor.to(args.device) for input_tensor in batch if isinstance(input_tensor, torch.Tensor))
    input_ids, token_type_ids, mc_token_ids, _, mc_labels = batch
    all_mc_logits = []
    for index in range(0, input_ids.size(1), candidates_per_forward):
        model_outputs = model(
            input_ids=input_ids[0, index:index+candidates_per_forward].unsqueeze(1),
        )
        mc_logits = model_outputs[0]
        all_mc_logits.append(mc_logits.detach())
    all_mc_logits = torch.cat(all_mc_logits, dim=0).unsqueeze(0)
    return torch.tensor(0.0), torch.tensor([]), all_mc_logits, mc_labels

### Start training!

In [15]:
set_seed(args)

In [16]:
config = AutoConfig.from_pretrained(args.model_name_or_path)
config.output_past = False
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model_class = BertForMultipleChoice
model = model_class.from_pretrained(args.model_name_or_path, config=config)
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Embedding(30529, 768)

In [17]:
model.to(args.device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30529, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [18]:
global_step, tr_loss = train(args, train_dataset, eval_dataset, model, tokenizer, 
      run_batch_selection_train, run_batch_selection_eval)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jennybae/miniconda3/envs/dstc9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Epoch:  50%|█████     | 1/2 [13:16<13:16, 796.25s/it]

Epoch: 100%|██████████| 2/2 [26:11<00:00, 785.73s/it]


## Evaluation

In [19]:
args.eval_only=""
args.eval_all_snippets = ""
args.labels_file = "data/val/labels.json"
args.checkpoint = "outputs/selection/checkpoint-1199"
model = model_class.from_pretrained(args.checkpoint)
tokenizer = AutoTokenizer.from_pretrained(args.checkpoint)

In [20]:
model.to(args.device)

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30529, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [21]:
eval_dataset = KnowledgeSelectionDataset(dataset_args, tokenizer, split_type="val", 
                             labels=not args.no_labels, labels_file=args.labels_file)
result = evaluate(args, eval_dataset, model, tokenizer, 
                  run_batch_selection_eval, desc=args.eval_desc or "val")

In [22]:
1

1

In [24]:
result

{'loss': 0.0, 'accuracy': 0.76}